<p><font size="6"><b>Pandas: Combining datasets Part I - concat</b></font></p>

> *DS Data manipulation, analysis and visualisation in Python*  
> *December, 2017*

> *© 2016, Joris Van den Bossche and Stijn Van Hoey  (<mailto:jorisvandenbossche@gmail.com>, <mailto:stijnvanhoey@gmail.com>). Licensed under [CC BY 4.0 Creative Commons](http://creativecommons.org/licenses/by/4.0/)*

---

In [1]:
import pandas as pd

Combining data is essential functionality in a data analysis workflow. 

Data is distributed in multiple files, different information needs to be merged, new data is calculated, .. and needs to be added together. Pandas provides various facilities for easily combining together Series and DataFrame objects

In [2]:
# redefining the example objects

# series
population = pd.Series({'Germany': 81.3, 'Belgium': 11.3, 'France': 64.3, 
                        'United Kingdom': 64.9, 'Netherlands': 16.9})

# dataframe
data = {'country': ['Belgium', 'France', 'Germany', 'Netherlands', 'United Kingdom'],
        'population': [11.3, 64.3, 81.3, 16.9, 64.9],
        'area': [30510, 671308, 357050, 41526, 244820],
        'capital': ['Brussels', 'Paris', 'Berlin', 'Amsterdam', 'London']}
countries = pd.DataFrame(data)
countries

,area,capital,country,population
0,30510,Brussels,Belgium,11.3
1,671308,Paris,France,64.3
2,357050,Berlin,Germany,81.3
3,41526,Amsterdam,Netherlands,16.9
4,244820,London,United Kingdom,64.9


# Adding columns

As we already have seen before, adding a single column is very easy:

In [3]:
pop_density = countries['population']*1e6 / countries['area']

In [4]:
pop_density

0    370.370370
1     95.783158
2    227.699202
3    406.973944
4    265.092721
dtype: float64

In [5]:
countries['pop_density'] = pop_density

In [6]:
countries

,area,capital,country,population,pop_density
0,30510,Brussels,Belgium,11.3,370.370370
1,671308,Paris,France,64.3,95.783158
2,357050,Berlin,Germany,81.3,227.699202
3,41526,Amsterdam,Netherlands,16.9,406.973944
4,244820,London,United Kingdom,64.9,265.092721


Adding multiple columns at once is also possible. For example, the following method gives us a DataFrame of two columns:

In [7]:
countries["country"].str.split(" ", expand=True)

,0,1
0,Belgium,None
1,France,None
2,Germany,None
3,Netherlands,None
4,United,Kingdom


We can add both at once to the dataframe:

In [8]:
countries[['first', 'last']] = countries["country"].str.split(" ", expand=True)

In [9]:
countries

,area,capital,country,population,pop_density,first,last
0,30510,Brussels,Belgium,11.3,370.370370,Belgium,None
1,671308,Paris,France,64.3,95.783158,France,None
2,357050,Berlin,Germany,81.3,227.699202,Germany,None
3,41526,Amsterdam,Netherlands,16.9,406.973944,Netherlands,None
4,244820,London,United Kingdom,64.9,265.092721,United,Kingdom


# Concatenating data

The ``pd.concat`` function does all of the heavy lifting of combining data in different ways.

``pd.concat`` takes a list or dict of Series/DataFrame objects and concatenates them in a certain direction (`axis`) with some configurable handling of “what to do with the other axes”.


## Combining rows - ``pd.concat``

![](../img/schema-concat0.svg)

Assume we have some similar data as in `countries`, but for a set of different countries:

In [10]:
data = {'country': ['Nigeria', 'Rwanda', 'Egypt', 'Morocco', ],
        'population': [182.2, 11.3, 94.3, 34.4],
        'area': [923768, 26338 , 1010408, 710850],
        'capital': ['Abuja', 'Kigali', 'Cairo', 'Rabat']}
countries_africa = pd.DataFrame(data)
countries_africa 

,area,capital,country,population
0,923768,Abuja,Nigeria,182.2
1,26338,Kigali,Rwanda,11.3
2,1010408,Cairo,Egypt,94.3
3,710850,Rabat,Morocco,34.4


We now want to combine the rows of both datasets:

In [11]:
pd.concat([countries, countries_africa])

,area,capital,country,first,last,pop_density,population
0,30510,Brussels,Belgium,Belgium,None,370.370370,11.3
1,671308,Paris,France,France,None,95.783158,64.3
2,357050,Berlin,Germany,Germany,None,227.699202,81.3
3,41526,Amsterdam,Netherlands,Netherlands,None,406.973944,16.9
4,244820,London,United Kingdom,United,Kingdom,265.092721,64.9
0,923768,Abuja,Nigeria,NaN,NaN,NaN,182.2
1,26338,Kigali,Rwanda,NaN,NaN,NaN,11.3
2,1010408,Cairo,Egypt,NaN,NaN,NaN,94.3
3,710850,Rabat,Morocco,NaN,NaN,NaN,34.4


If we don't want the index to be preserved:

In [12]:
pd.concat([countries, countries_africa], ignore_index=True)

,area,capital,country,first,last,pop_density,population
0,30510,Brussels,Belgium,Belgium,None,370.370370,11.3
1,671308,Paris,France,France,None,95.783158,64.3
2,357050,Berlin,Germany,Germany,None,227.699202,81.3
3,41526,Amsterdam,Netherlands,Netherlands,None,406.973944,16.9
4,244820,London,United Kingdom,United,Kingdom,265.092721,64.9
5,923768,Abuja,Nigeria,NaN,NaN,NaN,182.2
6,26338,Kigali,Rwanda,NaN,NaN,NaN,11.3
7,1010408,Cairo,Egypt,NaN,NaN,NaN,94.3
8,710850,Rabat,Morocco,NaN,NaN,NaN,34.4


When the two dataframes don't have the same set of columns, by default missing values get introduced:

In [13]:
pd.concat([countries, countries_africa[['country', 'capital']]], ignore_index=True)

,area,capital,country,first,last,pop_density,population
0,30510.0,Brussels,Belgium,Belgium,None,370.370370,11.3
1,671308.0,Paris,France,France,None,95.783158,64.3
2,357050.0,Berlin,Germany,Germany,None,227.699202,81.3
3,41526.0,Amsterdam,Netherlands,Netherlands,None,406.973944,16.9
4,244820.0,London,United Kingdom,United,Kingdom,265.092721,64.9
5,NaN,Abuja,Nigeria,NaN,NaN,NaN,NaN
6,NaN,Kigali,Rwanda,NaN,NaN,NaN,NaN
7,NaN,Cairo,Egypt,NaN,NaN,NaN,NaN
8,NaN,Rabat,Morocco,NaN,NaN,NaN,NaN


We can also pass a dictionary of objects instead of a list of objects. Now the keys of the dictionary are preserved as an additional index level:

In [14]:
pd.concat({'europe': countries, 'africa': countries_africa})

area    capital         country        first     last  \
africa 0   923768      Abuja         Nigeria          NaN      NaN   
       1    26338     Kigali          Rwanda          NaN      NaN   
       2  1010408      Cairo           Egypt          NaN      NaN   
       3   710850      Rabat         Morocco          NaN      NaN   
europe 0    30510   Brussels         Belgium      Belgium     None   
       1   671308      Paris          France       France     None   
       2   357050     Berlin         Germany      Germany     None   
       3    41526  Amsterdam     Netherlands  Netherlands     None   
       4   244820     London  United Kingdom       United  Kingdom   

          pop_density  population  
africa 0          NaN       182.2  
       1          NaN        11.3  
       2          NaN        94.3  
       3          NaN        34.4  
europe 0   370.370370        11.3  
       1    95.783158        64.3  
       2   227.699202        81.3  
       3   406.973944        16.9  
       4   265.092721        64.9

## Combining columns  - ``pd.concat`` with ``axis=1``

![](../img/schema-concat1.svg)

Assume we have another DataFrame for the same countries, but with some additional statistics:

In [15]:
data = {'country': ['Belgium', 'France', 'Netherlands'],
        'GDP': [496477, 2650823, 820726],
        'area': [8.0, 9.9, 5.7]}
country_economics = pd.DataFrame(data).set_index('country')
country_economics

,GDP,area
country,,
Belgium,496477,8.0
France,2650823,9.9
Netherlands,820726,5.7


In [16]:
pd.concat([countries, country_economics], axis=1)

,area,capital,country,population,pop_density,first,last,GDP,area
0,30510.0,Brussels,Belgium,11.3,370.370370,Belgium,None,NaN,NaN
1,671308.0,Paris,France,64.3,95.783158,France,None,NaN,NaN
2,357050.0,Berlin,Germany,81.3,227.699202,Germany,None,NaN,NaN
3,41526.0,Amsterdam,Netherlands,16.9,406.973944,Netherlands,None,NaN,NaN
4,244820.0,London,United Kingdom,64.9,265.092721,United,Kingdom,NaN,NaN
Belgium,NaN,NaN,NaN,NaN,NaN,NaN,NaN,496477.0,8.0
France,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2650823.0,9.9
Netherlands,NaN,NaN,NaN,NaN,NaN,NaN,NaN,820726.0,5.7


`pd.concat` matches the different objects based on the index:

In [17]:
countries2 = countries.set_index('country')

In [18]:
countries2

,area,capital,population,pop_density,first,last
country,,,,,,
Belgium,30510,Brussels,11.3,370.370370,Belgium,None
France,671308,Paris,64.3,95.783158,France,None
Germany,357050,Berlin,81.3,227.699202,Germany,None
Netherlands,41526,Amsterdam,16.9,406.973944,Netherlands,None
United Kingdom,244820,London,64.9,265.092721,United,Kingdom


In [19]:
pd.concat([countries2, country_economics], axis=1)

,area,capital,population,pop_density,first,last,GDP,area
Belgium,30510,Brussels,11.3,370.370370,Belgium,None,496477.0,8.0
France,671308,Paris,64.3,95.783158,France,None,2650823.0,9.9
Germany,357050,Berlin,81.3,227.699202,Germany,None,NaN,NaN
Netherlands,41526,Amsterdam,16.9,406.973944,Netherlands,None,820726.0,5.7
United Kingdom,244820,London,64.9,265.092721,United,Kingdom,NaN,NaN


# Joining data with `pd.merge`

Using `pd.concat` above, we combined datasets that had the same columns or the same index values. But, another typical case if where you want to add information of second dataframe to a first one based on one of the columns. That can be done with [`pd.merge`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html).

Let's look again at the titanic passenger data, but taking a small subset of it to make the example easier to grasp:

In [20]:
df = pd.read_csv("../data/titanic.csv")
df = df.loc[:9, ['Survived', 'Pclass', 'Sex', 'Age', 'Fare', 'Embarked']]

In [21]:
df

,Survived,Pclass,Sex,Age,Fare,Embarked
0,0,3,male,22.0,7.2500,S
1,1,1,female,38.0,71.2833,C
2,1,3,female,26.0,7.9250,S
3,1,1,female,35.0,53.1000,S
4,0,3,male,35.0,8.0500,S
5,0,3,male,NaN,8.4583,Q
6,0,1,male,54.0,51.8625,S
7,0,3,male,2.0,21.0750,S
8,1,3,female,27.0,11.1333,S
9,1,2,female,14.0,30.0708,C


Assume we have another dataframe with more information about the 'Embarked' locations:

In [22]:
locations = pd.DataFrame({'Embarked': ['S', 'C', 'Q', 'N'],
                          'City': ['Southampton', 'Cherbourg', 'Queenstown', 'New York City'],
                          'Country': ['United Kindom', 'France', 'Ireland', 'United States']})

In [23]:
locations

,City,Country,Embarked
0,Southampton,United Kindom,S
1,Cherbourg,France,C
2,Queenstown,Ireland,Q
3,New York City,United States,N


We now want to add those columns to the titanic dataframe, for which we can use `pd.merge`, specifying the column on which we want to merge the two datasets:

In [24]:
pd.merge(df, locations, on='Embarked', how='left')

,Survived,Pclass,Sex,Age,Fare,Embarked,City,Country
0,0,3,male,22.0,7.2500,S,Southampton,United Kindom
1,1,1,female,38.0,71.2833,C,Cherbourg,France
2,1,3,female,26.0,7.9250,S,Southampton,United Kindom
3,1,1,female,35.0,53.1000,S,Southampton,United Kindom
4,0,3,male,35.0,8.0500,S,Southampton,United Kindom
5,0,3,male,NaN,8.4583,Q,Queenstown,Ireland
6,0,1,male,54.0,51.8625,S,Southampton,United Kindom
7,0,3,male,2.0,21.0750,S,Southampton,United Kindom
8,1,3,female,27.0,11.1333,S,Southampton,United Kindom
9,1,2,female,14.0,30.0708,C,Cherbourg,France


In this case we use `how='left` (a "left join") because we wanted to keep the original rows of `df` and only add matching values from `locations` to it. Other options are 'inner', 'outer' and 'right' (see the [docs](http://pandas.pydata.org/pandas-docs/stable/merging.html#brief-primer-on-merge-methods-relational-algebra) for more on this).